In [1]:
import pandas as pd
import numpy as np
from phemap import Phemap
from tqdm import tqdm
import pickle
import matplotlib.pyplot as plt
import matplotlib as mpl
import scipy.stats
import seaborn as sns
import math

In [2]:
source_file = 'data/phecode_definitions1.2.csv'
mapping_file = 'data/phecode_map.csv'
phemap = Phemap(source_file=source_file, mapping_file=mapping_file)

Read patients phenotype

Field IDField title

41270 Diagnoses - ICD10
I'll take just field Diagnoses - ICD10 [41270-0.0 to 41270-0.222]<br>

41280 Date - ICD10
I'll take date - ICD10 [41280-0.0 to 41280-0.222]<br>


In [3]:
read_cols = ['eid']
for i in range(0,223):
    read_cols.append(f'41270-0.{i}')
    read_cols.append(f'41280-0.{i}')

In [4]:
df = pd.read_csv("I:/UKB_DATA/main_df/ukb47615.csv", usecols=read_cols)

C:\Users\Remote\AppData\Local\Temp\ipykernel_22416\1429465758.py:1: DtypeWarning: Columns (4105,4106,4107,4108,4109,4110,4111,4112,4113,4114,4115,4116,4117,4118,4119,4120,4121,4122,4123,4124,4125,4126,4127,4128,4129,4130,4131,4132,4133,4134,4135,4136,4137,4138,4139,4140,4141,4142,4143,4144,4145,4146,4147,4148,4149,4150,4151,4152,4153,4154,4155,4156,4157,4158,4159,4160,4161,4162,4163,4164,4165,4166,4167,4168,4169,4170,4171,4172,4173,4174,4175,4176,4177,4178,4179,4180,4181,4182,4183,4184,4185,4186,4187,4188,4189,4190,4191,4192,4193,4194,4195,4196,4197,4198,4199,4200,4201,4202,4203,4204,4205,4206,4207,4208,4209,4210,4211,4212,4213,4214,4215,4216,4217,4218,4219,4220,4221,4222,4223,4224,4225,4226,4227,4228,4229,4230,4231,4232,4233,4234,4235,4236,4237,4238,4239,4240,4241,4242,4243,4244,4245,4246,4247,4248,4249,4250,4251,4252,4253,4254,4255,4256,4257,4258,4259,4260,4261,4262,4263,4264,4265,4266,4267,4268,4269,4270,4271,4272,4273,4274,4275,4276,4277,4278,4279,4280,4281,4282,4283,4284,4285,4286

In [5]:
df_date = pd.read_csv("K:/ECG_labels.csv")[['eid','ekg_date']]

In [6]:
df_date['ekg_date']=[datetime.split(" ")[0] for datetime in list(df_date['ekg_date'])]

Take ecgs only that happened before ekg was taken

In [7]:
df = df.merge(df_date, on='eid')

In [8]:
df[f'41280-0.0']

0        2017-07-13
1        2015-05-14
2        2006-01-06
3        2013-09-19
4        2000-11-27
            ...    
44827    2012-10-18
44828    1998-09-24
44829    2019-01-29
44830    2005-07-31
44831    2017-09-18
Name: 41280-0.0, Length: 44832, dtype: object

In [10]:
df['ekg_date']

0        2016-11-03
1        2019-06-26
2        2019-11-25
3        2019-09-11
4        2017-05-09
            ...    
44827    2018-10-05
44828    2018-07-06
44829    2018-07-22
44830    2019-07-12
44831    2018-03-23
Name: ekg_date, Length: 44832, dtype: object

In [11]:
df[f'41280-0.0']>df['ekg_date']

0         True
1        False
2        False
3        False
4        False
         ...  
44827    False
44828    False
44829     True
44830    False
44831    False
Length: 44832, dtype: bool

In [12]:
from datetime import datetime

date1 = df[f'41280-0.0'][1]
date2 = df['ekg_date'][0]

date1 = datetime.strptime(date1, "%Y-%m-%d")
date2 = datetime.strptime(date2, "%Y-%m-%d")

diff = date2 - date1
print(diff)

-252 days, 0:00:00


In [8]:
for i in range(0,223):
    # is icd code assigned after ekg was taken
    df[f'41280-0.{i}']>df['ekg_date']
    df.loc[df[f'41280-0.{i}']>df['ekg_date'],f'41270-0.{i}']= np.nan

In [9]:
icd9_codes={'eid': np.array(df['eid'])}
for i in range(0,223):
    icd9_codes[f'ICD10_{i+1}']= np.array(df[f'41270-0.{i}'])

In [10]:
df_icd9=pd.DataFrame(icd9_codes)

In [11]:
df_icd9

,eid,ICD10_1,ICD10_2,ICD10_3,ICD10_4,ICD10_5,ICD10_6,ICD10_7,ICD10_8,ICD10_9,...,ICD10_214,ICD10_215,ICD10_216,ICD10_217,ICD10_218,ICD10_219,ICD10_220,ICD10_221,ICD10_222,ICD10_223
0,1000012,C504,C780,C782,NaN,NaN,J450,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000158,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1000243,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1000262,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1000369,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44827,6024225,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44828,6024290,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44829,6024624,K644,K649,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44830,6024638,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
df_icd9.to_csv("ecg_icd9.csv")

## Mapping ICD9 codes to PheCodes

In [13]:
df_icd9=pd.read_csv("ecg_icd9.csv")

C:\Users\Remote\AppData\Local\Temp\ipykernel_13760\4126539325.py:1: DtypeWarning: Columns (52,56,57,66,67,68,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,100,101,103,104,105,106,109,111,112,113,114,116,117,120,123,130,131,132,137,141,142,148,153,155) have mixed types. Specify dtype option on import or set low_memory=False.
  df_icd9=pd.read_csv("ecg_icd9.csv")


In [14]:
source_file = 'data/phecode_definitions1.2.csv'
mapping_file = 'data/phecode_map.csv'

In [15]:
mapping_df = pd.read_csv(mapping_file)

In [16]:
phemap = Phemap(source_file=source_file, mapping_file=mapping_file)

In [17]:
patient_diag = {}
# set keys to eids
for eid in df_icd9.eid:
    patient_diag[eid]= []

In [18]:
from tqdm import tqdm

In [19]:
for idx, row in tqdm(df_icd9.iterrows()):
    for i in range(1,224):
        # checks if not nan and if mapping key exists
        if row[f"ICD10_{i}"]==row[f"ICD10_{i}"] and row[f"ICD10_{i}"] in list(mapping_df.ICD10):
            patient_diag[row['eid']] = patient_diag[row['eid']] + phemap.get_phecode_for_icd10(row[f"ICD10_{i}"])


44832it [03:02, 245.06it/s]


In [20]:
patient_diag

{1000012: ['174.11',
  '198.2',
  '198.2',
  '495.0',
  '939.0',
  '733.0',
  '850.0',
  '1015.0',
  '965.1',
  '175.0'],
 1000158: [],
 1000243: [],
 1000262: [],
 1000369: [],
 1000489: ['364.4', '366.2', '367.2'],
 1000490: [],
 1000542: [],
 1000563: [],
 1000670: ['172.2'],
 1000816: ['38.0',
  '994.2',
  '480.3',
  '270.32',
  '270.32',
  '276.41',
  '276.13',
  '443.1',
  '480.11',
  '509.2',
  '506.0',
  '714.0',
  '714.1',
  '716.9',
  '580.2',
  '585.1',
  '585.3',
  '1011.0',
  '306.0',
  '960.2'],
 1001023: [],
 1001100: ['550.1', '716.9', '716.2'],
 1001231: [],
 1001363: ['244.4', '480.11', '512.8', '514.0', '519.8'],
 1001483: ['562.1', '785.0'],
 1001525: ['733.0', '608.0', '306.0'],
 1001614: ['189.21', '278.1', '401.1', '600.0', '751.22', '1019.0', '306.0'],
 1001620: ['401.1', '550.1', '689.0'],
 1001993: [],
 1002013: [],
 1002097: [],
 1002291: [],
 1002347: ['285.0',
  '415.0',
  '451.2',
  '452.0',
  '454.0',
  '550.2',
  '563.0',
  '760.0',
  '585.1',
  '586.2',

In [21]:
filehandler = open("pheno_patient_diag_ecg", 'wb')
pickle.dump(patient_diag, filehandler)

## Convert to hot encoding phenocode

In [22]:
with open(r"pheno_patient_diag_ecg", "rb") as input_file:
    non_imaging = pickle.load(input_file)

In [23]:
# Get corresponding eids from ECG
import os

ekg_names = [file_name for file_name in os.listdir("K:/UKBB_EKG_NUMPY")]
eids = [int(file_name.split('_')[0]) for file_name in os.listdir("K:/UKBB_EKG_NUMPY")]

In [24]:
len(ekg_names)

44835

In [25]:
d_keys = [key for key in non_imaging.keys()]

In [26]:
# find if any ids appear in eeg files but not in a dictionary
to_remove = np.setdiff1d(eids,d_keys)

ekg_names = [ elem for elem in ekg_names if not int(elem.split("_")[0]) in to_remove]
eids = [ elem for elem in eids if not elem in to_remove]

In [27]:
# now keep only needed ids
phecode_dict = { eid: non_imaging[eid] for eid in eids }

In [28]:
# Create hot encodings
phecodes = [str(float(code['phecode'])) for code in phemap.get_all_phecodes()]
df = pd.DataFrame(0, index=ekg_names, columns=phecodes)

In [29]:
df

,8.0,8.5,8.51,8.52,8.6,8.7,10.0,31.0,31.1,38.0,...,986.0,987.0,988.0,989.0,990.0,994.0,994.1,994.2,994.21,996.0
1000012_20205_2_0.npy,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1000158_20205_2_0.npy,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1000243_20205_2_0.npy,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1000262_20205_2_0.npy,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1000369_20205_2_0.npy,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6024225_20205_2_0.npy,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6024290_20205_2_0.npy,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6024624_20205_2_0.npy,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6024638_20205_2_0.npy,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
for counter, eid in tqdm(enumerate(eids)):
    for phec in phecode_dict[eid]:
        #NOTE: WHEN PATIENT DIAGS WERE STORED dict values got converted from str to numbers
        # and that's why strings 31 got converted to 31.0 
        # this gives an error and we fix it temporarily with the next if statement
        # TODO: GO back and make sure patient diags never get converted to numbers (ints or floats)
        df.loc[ekg_names[counter], phec]=1

44832it [00:04, 10470.24it/s]


In [31]:
df.to_numpy()[:,1:].sum()

78664

In [32]:
df.insert(0,'Filename',np.array(list(df.index)))

In [33]:
check = pd.read_csv("K:/UKBB_all_phecode.csv")

In [34]:
df.index = np.arange(0,44832)

In [35]:
df.to_csv("K:/UKBB_before_ecg_phecode",index=0)

In [36]:
check

,Filename,8.0,8.5,8.51,8.52,8.6,8.7,10.0,31.0,31.1,...,986.0,987.0,988.0,989.0,990.0,994.0,994.1,994.2,994.21,996.0
0,1000012_20205_2_0.npy,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1000158_20205_2_0.npy,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1000243_20205_2_0.npy,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1000262_20205_2_0.npy,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1000369_20205_2_0.npy,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44827,6024225_20205_2_0.npy,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
44828,6024290_20205_2_0.npy,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
44829,6024624_20205_2_0.npy,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
44830,6024638_20205_2_0.npy,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
df

,Filename,8.0,8.5,8.51,8.52,8.6,8.7,10.0,31.0,31.1,...,986.0,987.0,988.0,989.0,990.0,994.0,994.1,994.2,994.21,996.0
0,1000012_20205_2_0.npy,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1000158_20205_2_0.npy,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1000243_20205_2_0.npy,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1000262_20205_2_0.npy,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1000369_20205_2_0.npy,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44827,6024225_20205_2_0.npy,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
44828,6024290_20205_2_0.npy,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
44829,6024624_20205_2_0.npy,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
44830,6024638_20205_2_0.npy,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
